In [1]:
from pinecone import Pinecone
from dotenv import load_dotenv
import os
import voyageai

load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("rag-chat")
VOYAGE_API_KEY = os.getenv('VOYAGE_API_KEY')
vo = voyageai.Client()

c:\Users\Home\anaconda3\envs\RAG-CHAT-DEV\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [57]:
query_text = "How does Sam Altman, the renowned entrepreneur and former President of Y Combinator, perceive the division of equity among co-founders in startup companies? Given his extensive experience in the startup ecosystem and his role in guiding numerous emerging companies, what insights or recommendations has Altman offered regarding how co-founders should approach the distribution of equity shares? Specifically, does he advocate for equal splits, or does he suggest a more performance-based allocation? What are his views on addressing potential discrepancies in co-founder contributions and the impact of such equity decisions on the overall dynamics and success of startup ventures?"
result = vo.embed(query_text, model="voyage-large-2", input_type="query")

In [58]:
vector = result.embeddings[0]

In [62]:
result = index.query(
    vector=vector,
    filter={
        "pdf_id": "default"
    },
    top_k=3,
    include_metadata=True
)

In [63]:
result

{'matches': [{'id': 'default16',
              'metadata': {'pdf_id': 'default',
                           'text': 'Consider these criteria when you’re '
                                   'choosing a cofounder -- it’s one of the '
                                   'most important\n'
                                   'decisions you’ll make, and it’s often done '
                                   'fairly randomly. You want someone you know '
                                   'well,\n'
                                   'not someone you just met at a cofounder '
                                   'dating thing. You can evaluate anyone you '
                                   'might\n'
                                   'work with better with more data, and you '
                                   'really don’t want to get this one wrong. '
                                   'Also, at\n'
                                   'some point, the expected value of the '
                     

In [64]:
text_strings = [i["metadata"]["text"] for i in result["matches"]]
print(text_strings)

['Consider these criteria when you’re choosing a cofounder -- it’s one of the most important\ndecisions you’ll make, and it’s often done fairly randomly. You want someone you know well,\nnot someone you just met at a cofounder dating thing. You can evaluate anyone you might\nwork with better with more data, and you really don’t want to get this one wrong. Also, at\nsome point, the expected value of the startup is likely to dip below the X axis. If you have a\npre-existing relationship with your cofounders, none of you will want to let the other down and\nyou’ll keep going. Cofounder breakups are one of the leading causes of death for early\nstartups, and we see them happen very, very frequently in cases where the founders met for\nthe express purpose of starting the company. The best case, by far, is to have a good cofounder. The next best is to be a solo founder. The\nworse case, by far, is to have a bad cofounder. If things are not working out, you should part\nways quickly. A quick 

In [28]:
import dspy
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
gpt4 = dspy.OpenAI(model="gpt-4", api_key=OPENAI_API_KEY)
dspy.configure(lm=gpt4)
client = OpenAI()

class QueryReformatter(dspy.Signature):
    """Given a query, make it more detailed by asking implied subquestions for a vector search."""

    query = dspy.InputField()
    new_query = dspy.OutputField(desc="The more detailed version of the query, assuming it is known the information is contained in the writing. ONLY GIVE THE QUERY, no additional text.")
queryref = dspy.Predict(QueryReformatter)

In [39]:
from typing import List, Union, Optional

class PineconeRM(dspy.Retrieve):
    def __init__(self, id:str = "", k:int = 3):
        super().__init__(k=k)
        self.id = id

    def forward(self, query:str) -> dspy.Prediction:
        queryref = dspy.Predict(QueryReformatter)
        query_redone = queryref(query=query).new_query
        voyage_call = vo.embed(query_redone, model="voyage-large-2", input_type="query")
        query_vector = voyage_call.embeddings[0]
        if not self.id:
            result = index.query(
                vector=query_vector,
                top_k=self.k,
                include_metadata=True
            )
        else:
            result = index.query(
                vector=query_vector,
                filter={
                    "pdf_id": self.id
                },
                top_k=self.k,
                include_metadata=True
            )
        text_strings = [i["metadata"]["text"] for i in result["matches"]]
        return dspy.Prediction(
            passages=text_strings
        )

In [40]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with as ground-truth information as possible."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField()

class RAG(dspy.Module):
    def __init__(self, num_passages=3, id:str = ""):
        super().__init__()

        self.retrieve = PineconeRM(id=id, k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [41]:
dspy.configure(lm=gpt4, rm=PineconeRM)

In [42]:
rag = RAG(id="default")

In [43]:
response = rag(question="How does Sam Altman perceive the division of equity among co-founders in startup companies?")

In [44]:
response

Prediction(
    context=['Consider these criteria when you’re choosing a cofounder -- it’s one of the most important\ndecisions you’ll make, and it’s often done fairly randomly. You want someone you know well,\nnot someone you just met at a cofounder dating thing. You can evaluate anyone you might\nwork with better with more data, and you really don’t want to get this one wrong. Also, at\nsome point, the expected value of the startup is likely to dip below the X axis. If you have a\npre-existing relationship with your cofounders, none of you will want to let the other down and\nyou’ll keep going. Cofounder breakups are one of the leading causes of death for early\nstartups, and we see them happen very, very frequently in cases where the founders met for\nthe express purpose of starting the company. The best case, by far, is to have a good cofounder. The next best is to be a solo founder. The\nworse case, by far, is to have a bad cofounder. If things are not working out, you should part